## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = path.join(getcwd(), "AES_Flash-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\AES_Flash\\AES_Flash-S'

In [6]:
atprogram(project_path, verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:3633
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x000045c8.
[DEBUG] Memory segment base written at 0x000045c8. Size = 0x00000068.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

Create a figure for the plot.

In [7]:
fig = plt.figure(figsize=(10, 6))
fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_values": [True, True, True, True],
    "gpio_delay_time" : 0.0007,
    "plot_pins_method": "line",
    "plot_xmax": 100,
    "fig": fig,
    "window_title": "Experiment AES Flash",
    "file_name_base": "experiment_aes_flash"
}

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [11]:
data = []
fig.clf()
for ax in fig.get_axes():
    ax.cla()

with DGILibExtra(**config_dict) as dgilib:
    dgilib.device_reset()
    dgilib.logger.log(100,stop_fn)
    data = dgilib.data

## Analysis

In [12]:
aes_charge, aes_time = power_and_time_per_pulse(data, 2, 0.01)

In [13]:
flash_charge, flash_time = power_and_time_per_pulse(data, 3, 0.01)

In [14]:
aes_encrypt_charge = aes_charge[0::2]
aes_decrypt_charge = aes_charge[1::2]
aes_encrypt_time = aes_time[0::2]
aes_decrypt_time = aes_time[1::2]

In [15]:
flash_write_charge = flash_charge[0::2]
flash_read_charge = flash_charge[1::2]
flash_write_time = flash_time[0::2]
flash_read_time = flash_time[1::2]

In [16]:
MBEDTLS_AES_BLOCK_SIZE = 16
MIN_AES_BLOCKS = 1
num_bytes = range(MIN_AES_BLOCKS * MBEDTLS_AES_BLOCK_SIZE, MBEDTLS_AES_BLOCK_SIZE * (MIN_AES_BLOCKS + len(aes_encrypt_charge)), MBEDTLS_AES_BLOCK_SIZE)
print(f"MAX_AES_BLOCKS: {MIN_AES_BLOCKS + len(aes_encrypt_charge) - 1}")

MAX_AES_BLOCKS: 100


In [17]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [18]:
results = []
for y in [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 1.1344e-13
    reduced chi-square = 1.1575e-15
    Akaike info crit   = -3437.26850
    Bayesian info crit = -3432.05816
[[Variables]]
    slope:      3.2414e-08 +/- 7.3664e-12 (0.02%) (init = 0)
    intercept: -1.5299e-08 +/- 6.8558e-09 (44.81%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.868

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 100
    # variables        = 2
    chi-square         = 1.3168e-13
    reduced chi-square = 1.3437e-15
    Akaike info crit   = -3422.35867
    Bayesian info crit = -3417.14833
[[Variables]]
    slope:      3.3401e-08 +/- 7.9367e-12 (0.02%) (init = 0)
    intercept: -1.8759e-08 +/- 7.3865e-09 (39.38%) (init = 1)
[[Correlations]] (unrep

In [19]:
fig2 = plt.figure(figsize=(9, 8))
fig2.canvas.set_window_title("Analysis AES Flash")

In [20]:
charge_color = 'r'
time_color = 'b'

In [21]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [mC]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [22]:
lines = []
lines += ax1.plot(num_bytes, aes_encrypt_charge, charge_color+'-', label='AES Encrypt Charge')
lines += ax1.plot(num_bytes, aes_decrypt_charge, charge_color+'--', label='AES Decrypt Charge')
lines += ax2.plot(num_bytes, aes_encrypt_time, time_color+'-', label='AES Encrypt Time')
lines += ax2.plot(num_bytes, aes_decrypt_time, time_color+'--', label='AES Decrypt Time')
lines += ax1.plot(num_bytes, flash_write_charge, charge_color+'-.', label='Flash Write Charge')
lines += ax1.plot(num_bytes, flash_read_charge, charge_color+':', label='Flash Read Charge')
lines += ax2.plot(num_bytes, flash_write_time, time_color+'-.', label='Flash Write Time')
lines += ax2.plot(num_bytes, flash_read_time, time_color+':', label='Flash Read Time')
ax1.legend(handles=lines)
# [aes_encrypt_charge, aes_decrypt_charge, aes_encrypt_time, aes_decrypt_time, flash_write_charge, flash_read_charge, flash_write_time, flash_read_time]:
ax1.set_title(f"AES Encrypt Charge: Base {results[0].params['intercept'].value:.03} mC plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"AES Decrypt Charge: Base {results[1].params['intercept'].value:.03} mC plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"AES Encrypt Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"AES Decrypt Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n" +
             f"Flash Write Charge: Base {results[4].params['intercept'].value:.03} mC plus {results[4].params['slope'].value:.03} mC per byte\n" +
             f"Flash Read Charge: Base {results[5].params['intercept'].value:.03} mC plus {results[5].params['slope'].value:.03} mC per byte\n" +
             f"Flash Write Time: Base {results[6].params['intercept'].value:.03} s plus {results[6].params['slope'].value:.03} s per byte\n" +
             f"Flash Read Time: Base {results[7].params['intercept'].value:.03} s plus {results[7].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [24]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(aes_encrypt_charge, open("aes_flash_encrypt_charge.p", "wb"))
pickle.dump(aes_decrypt_charge, open("aes_flash_decrypt_charge.p", "wb"))
pickle.dump(flash_write_charge, open("aes_flash_write_charge.p", "wb"))
pickle.dump(flash_read_charge, open("aes_flash_read_charge.p", "wb"))